# Getting started with Layer

[![Open in Layer](https://app.layer.ai/assets/badge.svg)](https://app.layer.ai/layer/titanic) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/titanic/Getting_Started_With_Layer.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/titanic)

Layer helps you build, train and track all your machine learning project metadata including ML models and datasets‍ with semantic versioning, extensive artifact logging and dynamic reporting with local↔cloud training.

In this quick walkthrough, we will see how to register and track our model with Layer.

## Installation

Let's start with installing [Layer](https://layer.ai).

In [ ]:
!pip install -U layer -q

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from layer.decorators import model
import seaborn as sns
import xgboost as xgb
import pandas as pd
import layer

## Inititialize Your First Layer Project

It's time to create your first [Layer Project](https://docs.app.layer.ai/docs/projects/layer-projects). 

Layer projects are basically containers to organize your machine learning project assets. They associate some combination of datasets and models. Layer projects are basically the front page of your ML projects.

In [ ]:
# Log in to your free account!
layer.login()

In [ ]:
layer.init("getting-started")

## Train and Register Your Model

We will be training a `RandomForestClassifier` to predict the survivors. As you can see the following function is a simple training function. We just added the [@model](https://docs.app.layer.ai/docs/sdk-library/model-decorator) decorator to integrate Layer into our pipeline

In [ ]:
@model("california_housing")
def train():
    X, y = fetch_california_housing(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(X_train, y_train)    
    predictions = xgb_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    
    layer.log({"mse": mse})
    return xgb_model

xgb_model = train()

In [ ]:
@model("california_housing")
def train():
    X, y = fetch_california_housing(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    xgb_model = xgb.XGBRegressor(max_depth=parameters["max_depth"], n_estimators=parameters["n_estimators"])
    xgb_model.fit(X_train, y_train)
    
    predictions = xgb_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    layer.log(parameters)
    layer.log({"mse": mse})
    return xgb_model

parameters = { "max_depth":2, "n_estimators":50}
xgb_model = train()

In [ ]:
@model("california_housing")
def train():
    X, y = fetch_california_housing(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    xgb_model = xgb.XGBRegressor(max_depth=parameters["max_depth"], n_estimators=parameters["n_estimators"])
    xgb_model.fit(X_train, y_train)
    
    predictions = xgb_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    
    table = pd.DataFrame(zip(predictions,y_test),columns=['Predicted Price', "Actual Price"])
    plot = sns.histplot((y_test-predictions), kde=True).figure
    layer.log({"Predictions":plot})
    layer.log({"Predictions vs Actual":table[:50]})
    layer.log(parameters)
    layer.log({"mse": mse})
    
    return xgb_model

parameters = { "max_depth":4, "n_estimators":100}
xgb_model = train()

In [ ]:
layer.run([train])

### Fetching an ML model from Layer

After you build the Layer Project, you can fetch your trained models from Layer with simple calling `layer.get_model("MODEL_NAME")`

If you [make your project public](https://docs.app.layer.ai/docs/projects/create-project), anyone even without a Layer account can fetch your models for their own use.

In [ ]:
xgb_model = layer.get_model("california_housing").get_train()

X, y = fetch_california_housing(return_X_y=True)
preds = xgb_model.predict(X)
preds[0]

## Where to go from here?

Now that you have created first Layer Project, you can:

- Join our [Slack Community ](https://bit.ly/layercommunityslack)
- Visit [Layer Examples Repo](https://github.com/layerai/examples) for more examples
- Browse [Trending Layer Projects](https://layer.ai) on our mainpage
- Check out [Layer Documentation](https://docs.layer.ai) to learn more